In [2]:
%pip install yfinance

  Using cached yfinance-0.2.9-py2.py3-none-any.whl (55 kB)
     |████████████████████████████████| 4.8 MB 2.9 MB/s eta 0:00:01
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2021.3
    Uninstalling pytz-2021.3:
      Successfully uninstalled pytz-2021.3
  Attempting uninstall: lxml
    Found existing installation: lxml 4.8.0
    Uninstalling lxml-4.8.0:
      Successfully uninstalled lxml-4.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import json

In [5]:
def getTickerData(ticker):
    # 2022 Superbowl Date & 1 year +
    start_date = '2022-02-08'
    end_date = '2023-02-08'
    
    # get data
    data = yf.download(ticker, start_date, end_date)
    
    # adjust df for plotting
    data['Date'] = data.index
    
    data.reset_index(drop=True, inplace=True)
    
    return data

def loadCommercialsJSON():
    f = open('../static/commercials.json')
    commercials = json.load(f)
    
    return commercials

def plotAndSaveImage(data, commercial_image):
    fig, ax = plt.subplots(figsize=(16,9))
    img = plt.imread('../static/' + commercial_image)
    
    start_price = data.iloc[0].Open
    end_price = data.iloc[len(data.index)-1].Close
    
    color = 'green' if end_price >= start_price else 'red'
    
    plt.plot(data['Date'], data['Open'], linewidth=20, color=color, alpha=0.8)
    fig.figimage(img, 0, 0, resize=True, zorder=-1)
    
    plt.tight_layout()
    plt.axis('off')
    plt.margins(x=0)
    plt.savefig(commercial_image.replace('.jpg', '') + '_graph.jpg', bbox_inches='tight', pad_inches=0)
#     plt.tight_layout()

def generateImagesOverlays():
    commercials = loadCommercialsJSON()

    for commercial in commercials['commercials']:
        ticker = commercial['ticker'].replace('$', '')
        commercial_image = commercial['imageLoc']

        tickerData = getTickerData(ticker)

        plotAndSaveImage(tickerData, commercial_image)

In [3]:
def getOneYearPerformance():
    commercials = loadCommercialsJSON()
    
    for commercial in commercials['commercials']:
        
        ticker = commercial['ticker'].replace('$', '')
        tickerData = getTickerData(ticker)
        
        start_price = tickerData.iloc[0].Open
        end_price = tickerData.iloc[len(tickerData.index)-1].Close
        
        yoy_return = 1000 * (end_price - start_price)/start_price
        print(ticker, start_price, end_price, yoy_return)